<a href="https://colab.research.google.com/github/gabrielborja/parc_de_montjuic/blob/main/game_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Game Analysis

## Uploading packages and data

In [ ]:
# Importing necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact

In [ ]:
# Setting the plot size
sns.set(rc = {'figure.figsize': (12, 5)})

In [ ]:
# Uploading google sheet data from drive
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc2 = gspread.authorize(creds)

# Loading dataset from google sheet
wb2 = gc2.open('sjakk_db')
ws2_data = wb2.sheet1
records_2 = ws2_data.get_all_records()

# Converting to a Pandas DataFrame
df1 = pd.DataFrame.from_records(records_2)
df1.tail(1)

In [ ]:
df1.info()

## Metrics calculation

In [ ]:
# Changing data types
df1['date'] = pd.to_datetime(df1['date'])
df1[['10sf','20sf','30sf','40sf','50sf','60sf']] = df1[['10sf','20sf','30sf','40sf','50sf','60sf']].apply(pd.to_numeric)
df1[['10tl','20tl','30tl','40tl','50tl','60tl']] = df1[['10tl','20tl','30tl','40tl','50tl','60tl']].apply(pd.to_numeric)

In [ ]:
# Generating datetime data
date_col = 'date' #-> Change to the desired column
def extract_datetime(df, date_col):
  df = df.assign(year = df[date_col].dt.year,
                 month = df[date_col].dt.month_name(),
                 weeknum = df[date_col].dt.isocalendar().week,
                 quarter = df[date_col].dt.to_period('Q').astype(str),
                 week_of_year = 'w' + df[date_col].dt.isocalendar().week.astype(str))
  return df

df1 = extract_datetime(df1, date_col)

In [ ]:
# Calculating move metrics
def calculate_moves(df):
  mov_t = round(df['moves'].median(), 1)
  mov_w = round(df[df['color'] == 'w']['moves'].median(), 1)
  mov_b = round(df[df['color'] == 'b']['moves'].median(), 1)
  df_mov = pd.DataFrame((mov_t, mov_w, mov_b), index=['total','white','black'], columns=['median_moves'])
  return df_mov

calculate_moves(df1)

In [ ]:
# Calculating win metrics
def calculate_victories(df):
  win_t = round(df1[df1['win'] == 1]['win'].sum()/df1.shape[0], 3)
  win_w = round(df1[(df1['win'] == 1) & (df1['color'] == 'w')]['win'].sum()/ (df1[df1['color'] == 'w']['win'].count()), 3)
  win_b = round(df1[(df1['win'] == 1) & (df1['color'] == 'b')]['win'].sum()/ (df1[df1['color'] == 'b']['win'].count()), 3)
  df_win = pd.DataFrame((win_t, win_w, win_b), index=['total','white','black'], columns=['win_perc'])
  return df_win

In [ ]:
# Calculating StockFish metrics
def calculate_stockfish_white(df):
  sto_10 = round(df1[df1['color'] == 'w']['10sf'].median(), 3)
  sto_20 = round(df1[df1['color'] == 'w']['20sf'].median(), 3)
  sto_30 = round(df1[df1['color'] == 'w']['30sf'].median(), 3)
  sto_40 = round(df1[df1['color'] == 'w']['40sf'].median(), 3)
  sto_50 = round(df1[df1['color'] == 'w']['50sf'].median(), 3)
  sto_60 = round(df1[df1['color'] == 'w']['60sf'].median(), 3)
  df_sto = pd.DataFrame((sto_10,sto_20,sto_30,sto_40,sto_50,sto_60), index=['sf_10','sf_20','sf_30','sf_40','sf_50','sf_60'], columns=['stockfish_eval'])
  return df_sto
calculate_stockfish_white(df1)

## Analysis Dashboard

In [ ]:
# Adding interactivity to graphs
# import ipywidgets as widgets
# from IPython import display
# from ipywidgets import interact, interactive, fixed, interact_manual

In [ ]:
yr = ['2024','2025']

In [ ]:
# Creating a function to plot the interactive dashboard
@interact(Year = df1['year'].unique())
def plot_analysis_dashboard(Year):
  """Plot the analysis of games"""

  # Creating four subplots and unpack the output array immediately
  fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots(nrows=2, ncols=4, figsize=(13,5))

  # Setting title for the figure
  fig.suptitle("GAD", fontsize=20)

  # Plotting multiple graphs
  ax1.plot(df1['rating'])
  ax2.bar(df_mov.index, df_mov['median_moves'], color=['b','lightgray','k']) #['cornflowerblue','lightsteelblue','black']
  ax3.bar(df_mov.index, df_win['win_perc'], color=['b','lightgray','k'])

  # Customizing axes labels and titles for first axes figure
  ax1.set_xlabel('Games')
  ax1.set_ylabel('Rating')
  ax1.set_title(f'Games Rat')

  # Prevent from axis to colliding or overlaping one another.
  fig.tight_layout()
  fig.subplots_adjust(top=0.88)